## 데이터셋 구축

In [58]:
import pandas as pd
import re

In [59]:
!pip install konlpy
from konlpy.tag import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [60]:
ner1_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AURA2/NER_df.csv')
ner2_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AURA2/NER2_df.csv')
ner3_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AURA2/NER3_df.csv')

In [61]:
ner_df = pd.concat([ner1_df, ner2_df, ner3_df])

In [62]:
ner_df.reset_index(inplace = True, drop = True)

In [63]:
id_tag = dict([])
for i in range(len(ner_df)) :
  id = ner_df.loc[i, 'id']
  if id not in id_tag :
    id_tag_tmp = dict([])
    ori_form = ner_df.loc[i, 'ori_form']
    ori_form_dic = dict([])
    ori_form_split = ori_form.split()
    for j in ori_form_split :
      if len(ori_form_dic) == 0 :
        ori_form_dic[j] = 'B-' + ner_df.loc[i,'label']
      else :
        ori_form_dic[j] = 'I-' + ner_df.loc[i,'label']
    id_tag[id] = ori_form_dic

  elif id in id_tag :
    tmp_dic = dict([])
    ori_form = ner_df.loc[i, 'ori_form']
    ori_form_split = ori_form.split()
    for j in range(len(ori_form_split)) :
      if j == 0 :
        id_tag[id][ori_form_split[j]] = 'B-' + ner_df.loc[i, 'label']
      else :
        id_tag[id][ori_form_split[j]] = 'I-' + ner_df.loc[i, 'label']

In [ ]:
id_list=list(id_tag.keys())
sentences = []
ner_tags = []
for i in range(len(id_list)) :
  id=id_list[i]
  key_list=list(id_tag[id].keys())
  string=ner_df[ner_df['id']==id]['sentence'].iloc[0]
  #print("초기 문장: ",string)
  #print("")
  #print(key_list)
  QWER_lst = []

  while(True) :
    tmp = 0
    for key in key_list:
        if key in string :
          QWER_lst.append(key)
          string = string.replace(key, 'QWER', 1)
          #print("hi")
          tmp = 1
    if tmp == 0 :
      break

  #print("QWER replace 후 문장: ",string)
  #print("QWER_lst : ", QWER_lst)
  #print("string : ", string)

    #komoran = Komoran()
  okt=Okt()
  a=list(okt.pos(string))
  #print(id_tag[id])
  #print(key_list)

  sen_result_list=[]
  tag_result_list=[]
  index=0
  for j in range(len(a)):
      if a[j][0]=='QWER':
          #print(j, i)
          #if index > (len(key_list)-1) :
          #  sen_result_list.append()
          sen_result_list.append(QWER_lst[index])
          tag_result_list.append(id_tag[id][QWER_lst[index]])
          index+=1
      else:
          sen_result_list.append(a[j][0])
          tag_result_list.append('O')
  
  sentences.append(sen_result_list)
  ner_tags.append(tag_result_list)
  #print(sen_result_list)
  #print(tag_result_list)
  #print(len(sen_result_list))
  #print(len(tag_result_list))

In [29]:
for i in range(len(sentences)) :
  if len(sentences[i]) != len(ner_tags[i]) :
    print('hi')

### 데이터 전처리 및 모델링

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [31]:
# 모든 단어를 사용하며 인덱스 1에는 단어 'OOV'를 할당.
src_tokenizer = Tokenizer(oov_token='OOV')
# 태깅 정보들은 내부적으로 대문자를 유지한 채 저장
tar_tokenizer = Tokenizer(lower=False)

src_tokenizer.fit_on_texts(sentences)
tar_tokenizer.fit_on_texts(ner_tags)

In [52]:
# 단어 집합 크기
vocab_size = len(src_tokenizer.word_index) + 1
# 태그 집합 크기
tag_size = len(tar_tokenizer.word_index) + 1
print(tag_size)
print(vocab_size)

12
92


In [33]:
# 정수 인코딩 시행
X_data = src_tokenizer.texts_to_sequences(sentences)
y_data = tar_tokenizer.texts_to_sequences(ner_tags)

In [34]:
print(X_data[0])
print(y_data[0])

[18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
[1, 1, 1, 5, 1, 2, 1, 1, 1, 1, 1, 1]


In [35]:
word_to_index = src_tokenizer.word_index
index_to_word = src_tokenizer.index_word
ner_to_index = tar_tokenizer.word_index
index_to_ner = tar_tokenizer.index_word

index_to_ner[0] = 'PAD'

In [36]:
print(max(len(w) for w in sentences))
print(min(len(w) for w in sentences))
print(np.array([len(w) for w in sentences]).mean())

38
12
22.2


In [37]:
max_len = 30 # max_len : 샘플의 토큰 길이의 최대를 보고 적당히 정한다.
X_data = pad_sequences(X_data, maxlen=max_len, padding = 'post')
y_data = pad_sequences(y_data, maxlen=max_len, padding = 'post') # 각 토큰에 대해 태깅을 하기에 maxlen이 같다

In [38]:
X_train, X_test, y_train_int, y_test_int = train_test_split(X_data, y_data, test_size = .2, random_state=777)

In [39]:
y_train = to_categorical(y_train_int, num_classes = tag_size)
y_test = to_categorical(y_test_int, num_classes = tag_size)

In [ ]:
!pip install keras-crf
!pip install seqeval
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, LSTM, Input, Bidirectional, TimeDistributed, Embedding, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras_crf import CRFModel
from seqeval.metrics import f1_score, classification_report

In [45]:
embedding_dim = 64
hidden_units = 32
dropout_ratio = 0.3

sequence_input = Input(shape=(max_len,),dtype=tf.int32, name='sequence_input')

model_embedding = Embedding(input_dim=vocab_size,
                            output_dim=embedding_dim,
                            input_length=max_len)(sequence_input)

model_bilstm = Bidirectional(LSTM(units=hidden_units, return_sequences=True))(model_embedding)

model_dropout = TimeDistributed(Dropout(dropout_ratio))(model_bilstm)

model_dense = TimeDistributed(Dense(tag_size, activation='relu'))(model_dropout)

base = Model(inputs=sequence_input, outputs=model_dense)
model = CRFModel(base, tag_size)
model.compile(optimizer=tf.keras.optimizers.Adam(0.001), metrics='accuracy')

In [47]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('bilstm_crf/cp.ckpt', monitor='val_decode_sequence_accuracy', mode='max', verbose=1, save_best_only=True, save_weights_only=True)

history = model.fit(X_train, y_train_int, batch_size=256, epochs=15, validation_split=0.1, callbacks=[mc, es])

Epoch 1/15
1/1 [==============================] - ETA: 0s - decode_sequence_accuracy: 0.6667 - loss: 69.3308
Epoch 1: val_decode_sequence_accuracy improved from -inf to 0.36667, saving model to bilstm_crf/cp.ckpt
1/1 [==============================] - 0s 156ms/step - decode_sequence_accuracy: 0.6667 - loss: 69.3308 - val_decode_sequence_accuracy: 0.3667 - val_loss: 66.0192
Epoch 2/15
1/1 [==============================] - ETA: 0s - decode_sequence_accuracy: 0.6667 - loss: 68.4415
Epoch 2: val_decode_sequence_accuracy did not improve from 0.36667
1/1 [==============================] - 0s 97ms/step - decode_sequence_accuracy: 0.6667 - loss: 68.4415 - val_decode_sequence_accuracy: 0.3667 - val_loss: 64.9788
Epoch 3/15
1/1 [==============================] - ETA: 0s - decode_sequence_accuracy: 0.6778 - loss: 67.1542
Epoch 3: val_decode_sequence_accuracy did not improve from 0.36667
1/1 [==============================] - 0s 93ms/step - decode_sequence_accuracy: 0.6778 - loss: 67.1542 - val_d

In [50]:
X_test[13]

IndexError: ignored

In [48]:
model.load_weights('bilstm_crf/cp.ckpt')

i = 13 # 확인하고 싶은 테스트용 샘플의 인덱스.
y_predicted = model.predict(np.array([X_test[i]]))[0] # 입력한 테스트용 샘플에 대해서 예측 y를 리턴
labels = np.argmax(y_test[i], -1) # 원-핫 인코딩을 다시 정수 인코딩으로 변경.

print("{:15}|{:5}|{}".format("단어", "실제값", "예측값"))
print(35 * "-")

for word, tag, pred in zip(X_test[i], labels, y_predicted[0]):
    if word != 0: # PAD값은 제외함.
        print("{:17}: {:7} {}".format(index_to_word[word], index_to_ner[tag], index_to_ner[pred]))

IndexError: ignored

In [ ]:
y_predicted = model.predict(X_test)[0]

In [ ]:
from seqeval.metrics import f1_score, classification_report

def sequences_to_tag(sequences):
    result = []
    # 전체 시퀀스로부터 시퀀스를 하나씩 꺼낸다.
    for sequence in sequences:
        word_sequence = []
        # 시퀀스로부터 확률 벡터 또는 원-핫 벡터를 하나씩 꺼낸다.
        for pred in sequence:
            # 정수로 변환. 예를 들어 pred가 [0, 0, 1, 0 ,0]라면 1의 인덱스인 2를 리턴한다.
            pred_index = np.argmax(pred)            
            # index_to_ner을 사용하여 정수를 태깅 정보로 변환. 'PAD'는 'O'로 변경.
            word_sequence.append(index_to_ner[pred_index].replace("PAD", "O"))
        result.append(word_sequence)
    return result

In [ ]:
def sequences_to_tag_for_crf(sequences): 
    result = []
    # 전체 시퀀스로부터 시퀀스를 하나씩 꺼낸다.
    for sequence in sequences: 
        word_sequence = []
        # 시퀀스로부터 예측 정수 레이블을 하나씩 꺼낸다.
        for pred_index in sequence:
            # index_to_ner을 사용하여 정수를 태깅 정보로 변환. 'PAD'는 'O'로 변경.
            word_sequence.append(index_to_ner[pred_index].replace("PAD", "O"))
        result.append(word_sequence)
    return result

pred_tags = sequences_to_tag_for_crf(y_predicted)
test_tags = sequences_to_tag(y_test)

print("F1-score: {:.1%}".format(f1_score(test_tags, pred_tags)))
print(classification_report(test_tags, pred_tags))